In [ ]:
import sys
sys.path.append('/code')

from database.models import (Protein, Organism, Classification, Molecule, Activity, ActivityType, Source, Quality, CID)

from sqlalchemy import create_engine
from sqlalchemy.orm import scoped_session
from sqlalchemy.orm import sessionmaker
import os
import glob
from rdkit import Chem
from razi.rdkit_postgresql.functions import morganbv_fp

import pandas as pd
# import modin.pandas as pd
import numpy as np
import json

import multiprocessing
import gc
from tqdm import tqdm
from copy import copy
import numpy as np

import dask.dataframe as dd


def get_db_session():
    engine = create_engine(
        os.environ.get('SQLALCHEMY_URL'), convert_unicode=True,
        pool_recycle=3600, pool_size=10)
    db_session = scoped_session(sessionmaker(
        autocommit=False, autoflush=False, bind=engine))
    
    return db_session


def get_or_create(session, model, **kwargs):
    instance = session.query(model).filter_by(**kwargs).first()
    if instance:
        return instance
    else:
        instance = model(**kwargs)
        session.add(instance)
        session.flush()
        session.refresh(instance)
        return instance
    
def get_or_instance(session, model, **kwargs):
    instance = session.query(model).filter_by(**kwargs).first()
    if instance:
        return (False, instance)
    else:
        instance = model(**kwargs)
        return (True, instance)
    
def sanitize_and_split(row, length, spl=';'):
    split = [v.rstrip() for v in str(row).split(spl)]
    if len(split)!= length:
        split = [split[0] for i in range(0,length)]
    
    split = [None if x == '' else x for x in split]
    
    return split


class TypeDecoder(json.JSONDecoder):
    """Custom json decoder to support types as values."""

    def __init__(self, *args, **kwargs):
        """Simple json decoder handling types as values."""
        json.JSONDecoder.__init__(self, object_hook=self.object_hook, *args, **kwargs)

    def object_hook(self, obj):
        """Handle types."""
        if '__type__' not in obj:
            return obj
        module = obj['__type__']['module']
        type_ = obj['__type__']['type']
        if module == 'builtins':
            return getattr(__builtins__, type_)
        loaded_module = importlib.import_module(module)
        return getattr(loaded_module, type_)
    


dtype_file = '../.data/papyrus/05.5/data_types.json'
activity_data = '../.data/papyrus/05.5/05.5_combined_set_without_stereochemistry.tsv.xz'
protein_data = '../.data/papyrus/05.5/05.5_combined_set_protein_targets.tsv.xz'


In [ ]:
with open(dtype_file, 'r') as jsonfile:
        dtypes = json.load(jsonfile, cls=TypeDecoder)['papyrus']

In [ ]:
protein_df = pd.read_csv(protein_data, sep='\t', dtype=dtypes)

In [ ]:
organisms = list(set(protein_df['Organism']))
classifications = []
for cstr in protein_df['Classification']:
    classifications.extend(str(cstr).split('->'))

classes = list(set(classifications))

In [ ]:
db_session = get_db_session()

rows = []

for i, row in protein_df.iterrows():
    organism = get_or_create(session=db_session, model=Organism, organism=row['Organism'])
    classifications_list = str(row['Classification']).split('->')
    classifications = [get_or_create(session=db_session, model=Classification, classification=c) for c in classifications_list]
    
    review_mapping = {'reviewed':1, 'Unreviewed':0, 'unreviewed':0}
    
    prot = Protein(
        target_id = row['target_id'],
        HGNC_symbol = str(row['HGNC_symbol']),
        uniprot_id = row['UniProtID'],
        reviewed = review_mapping[row['Status']],
        organism = organism.id,
        length = row['Length'],
        sequence = row['Sequence'], 
        classifications = classifications
    )
    
    rows.append(prot)
    
db_session.add_all(rows)
db_session.commit()
db_session.remove()
    

In [ ]:
def get_cid_arr(sources_list, cids_list, session):
    sources_cids_list = list(zip(sources_list, cids_list))
    cids = [      
        get_or_create(session=session, 
                      model=CID, 
                      cid=c[1], 
                      source=get_or_create(session=session, 
                                           model=Source, 
                                           source=c[0]).source) for c in sources_cids_list]
    return cids


def get_molecule(session, row_tup, cids):
#     out = []
    mol = Chem.MolFromSmiles(row_tup.SMILES)
    fp = morganbv_fp(row_tup.SMILES)

    # change this to use InChI and/or SMILES
    smiles = Chem.CanonSmiles(row_tup.SMILES)
    created, molecule = get_or_instance(session=session,model=Molecule,smiles=smiles,inchi=row_tup.InChI)

    for cid in cids:
        if cid not in molecule.cids:
            molecule.cids.append(cid)

    if created:
        molecule.smiles=smiles
        molecule.mol=mol
        molecule.inchi_key=row_tup.InChIKey
        molecule.inchi=row_tup.InChI
        molecule.inchi_auxinfo=row_tup.InChI_AuxInfo
        molecule.fp=fp
        molecule.connectivity=row_tup.connectivity
        session.add(molecule)
        session.flush()
        session.refresh(molecule)
        
    return molecule

def get_activity_dicts(row):
    slice_list = []
    if ';' in str(row.pchembl_value):

        pchembl_values = [v.rstrip() for v in row.pchembl_value.split(';')]
        length = len(pchembl_values)

        aids = sanitize_and_split(row=row.AID,length=length)        
        doc_ids = sanitize_and_split(row=row.all_doc_ids,length=length)
        years = sanitize_and_split(row=row.all_years,length=length)
        type_IC50s = sanitize_and_split(row=row.type_IC50,length=length)         
        type_EC50s = sanitize_and_split(row=row.type_EC50,length=length)
        type_KDs = sanitize_and_split(row=row.type_KD,length=length)
        type_Kis = sanitize_and_split(row=row.type_Ki,length=length)

        for j in range(0, len(pchembl_values)):
            update_dict = {
                'pchembl_value': pchembl_values[j],
                'AID': aids[j],
                'doc_id': doc_ids[j],
                'Year': years[j],
                'type_IC50': type_IC50s[j],
                'type_EC50': type_EC50s[j],
                'type_KD': type_KDs[j],
                'type_Ki': type_Kis[j]
            }
            row_copy = copy(row._asdict())

            row_copy.update(update_dict)

            slice_list.append(row_copy)

    else:
        slice_list.append(row._asdict())
            
    return slice_list

def process_row(row, session):
    
    activity_type_map = {
        '1000':'IC50',
        '0100':'EC50',
        '0010':'KD',
        '0001':'Ki',
        '0000':'other',
    }
    
    rows = []
    sources_list = row.source.split(';')
    cids_list = row.CID.split(';')
    cids = get_cid_arr(sources_list, cids_list, session)

    rows.extend(cids)

    molecule = get_molecule(session=session, row_tup=row, cids=cids)

    rows.append(molecule)

    qc,quality = get_or_instance(session=session, model=Quality, quality=row.Quality)
    if qc: rows.append(quality)
    qid = quality.id

    tc,target_id = get_or_instance(session=session, model=Protein, target_id=row.target_id)
    if tc: rows.append(target_id)
    tid = target_id.target_id

    molecule_id = molecule.id

    slice_list = get_activity_dicts(row)

    for s in slice_list:

        a = f"{s['type_IC50']}{s['type_EC50']}{s['type_KD']}{s['type_Ki']}"
        activity_type_str = activity_type_map[a]

        activity_type = get_or_create(session=session, model=ActivityType, type=activity_type_str).id

        try:
            y = int(s['Year'])
        except:
            y = None

        if str(s['doc_id']) in ['nan', 'NaN']:
            doc_id = None
        else:
            doc_id = str(s['doc_id'])

        activity = Activity(
            papyrus_activity_id=s['Activity_ID'],
            quality=qid,
            target_id=tid,
            molecule_id = molecule_id,
            accession=s['accession'],
            protein_type=s['Protein_Type'],
            aid = s['AID'],
            doc_id = doc_id,
            year = y,
            type = activity_type, 
            relation = s['relation'],
            pchembl_value = s['pchembl_value'],
            pchembl_value_mean = s['pchembl_value_Mean'],
            pchembl_value_stdev = s['pchembl_value_StdDev'],
            pchembl_value_SEM = s['pchembl_value_SEM'],
            pchembl_value_n = s['pchembl_value_N'],
            pchembl_value_median = s['pchembl_value_Median'],
            pchembl_value_mad = s['pchembl_value_MAD'],   
        )

        rows.append(activity)
        
    return rows
            

def process_activity_frame(df):
#     df_obj = df.select_dtypes(['object'])
    db_session = get_db_session()
    rows = []
    

    print('processing frame')
    
    
    
#     pool = multiprocessing.Pool(5)
#     results = pool.imap_unordered(process_row, list(zip([row for row in df.itertuples()], db_session)), chunksize=1000)

    ddf = dd.from_pandas(df, chunksize=1000)
    
    # do this bit in parallel?
    for row in tqdm(ddf.itertuples()):
        rows.extend(process_row(row, db_session))
        dd.compute()
            
    print('processing complete')
            
    del(df)
    db_session.add_all(rows)
    print('committing data')
    db_session.commit()
    db_session.close()
    db_session.remove()
    gc.collect()
            
    return True


In [ ]:
reader = pd.read_csv(activity_data, sep='\t', compression='xz', chunksize = 10000, iterator=True, dtype=dtypes)

for (i,df) in enumerate(reader):
    # process each data frame
    print(f'processing chunk {i}')
    process_activity_frame(df)
    gc.collect()


In [ ]:
import dask.dataframe as dd

ddata = dd.from_pandas(df)

In [ ]:
url = os.environ.get('SQLALCHEMY_URL')

In [ ]:
url

In [ ]:
import sys
sys.path.append('/code')

from database.models import (Protein, Organism, Classification, Molecule, Activity, ActivityType, Source, Quality, CID)

from sqlalchemy import create_engine
from sqlalchemy.orm import scoped_session
from sqlalchemy.orm import sessionmaker
import os
import glob

import pandas as pd
import numpy as np
import json

import multiprocessing
import gc
from tqdm import tqdm
from copy import copy
import numpy as np

from rdkit import Chem

def get_db_session():
    engine = create_engine(
        os.environ.get('SQLALCHEMY_URL'), convert_unicode=True,
        pool_recycle=3600, pool_size=10)
    db_session = scoped_session(sessionmaker(
        autocommit=False, autoflush=False, bind=engine))
    
    return db_session


class TypeDecoder(json.JSONDecoder):
    """Custom json decoder to support types as values."""

    def __init__(self, *args, **kwargs):
        """Simple json decoder handling types as values."""
        json.JSONDecoder.__init__(self, object_hook=self.object_hook, *args, **kwargs)

    def object_hook(self, obj):
        """Handle types."""
        if '__type__' not in obj:
            return obj
        module = obj['__type__']['module']
        type_ = obj['__type__']['type']
        if module == 'builtins':
            return getattr(__builtins__, type_)
        loaded_module = importlib.import_module(module)
        return getattr(loaded_module, type_)
    

# get neccessary molecule data as list of tuples
session = get_db_session()
mol_data = session.query(Molecule.id, Molecule.smiles, Molecule.inchi_auxinfo).all()
mol_ids = {}
for t in tqdm(mol_data):
    combo = f'{t[1]}_{t[2]}'
    if not combo in mol_ids.keys():
        mol_ids[combo] = [t[0]]
    else:
        mol_ids[combo].append(t[0])

# add activity types
types = ['EC50', 'IC50', 'KD', 'Ki', 'other']
# atypes = [ActivityType(type=t) for t in types]
# session.add_all(atypes)
# session.commit()

# return ids for activity types as dict
types = dict(session.query(ActivityType.type, ActivityType.id).all())

# add qualities
qualities = ["High","Low","Medium","Medium;Low","Low;Medium"]
# qs = [Quality(quality=q) for q in qualities]
# session.add_all(qs)
# session.commit()

# return ids for qualities as list of tuples
qualities = dict(session.query(Quality.quality,Quality.id).all())

# get all targets - probably don't need this 
targets = session.query(Protein.target_id).all()

pchembl_val_list = lambda pchembl_values: ([v.rstrip() for v in pchembl_values.split(';')])
smiles = lambda smi: (Chem.CanonSmiles(smi))

converters = {
    'SMILES':smiles,
    'pchembl_value':pchembl_val_list
}

dtype_file = '../.data/papyrus/05.5/data_types.json'
activity_data = '../.data/papyrus/05.5/05.5_combined_set_without_stereochemistry.tsv.xz'

with open(dtype_file, 'r') as jsonfile:
        dtypes = json.load(jsonfile, cls=TypeDecoder)['papyrus']

# all columns needed to process activity data
activity_columns = ['Activity_ID', 'SMILES', 'InChI_AuxInfo', 'accession', 'Protein_Type', 'AID', 'doc_id',
                    'Year', 'type_IC50', 'type_EC50', 'type_KD', 'type_Ki', 'relation', 'pchembl_value',
                    'pchembl_value_Mean', 'pchembl_value_StdDev', 'pchembl_value_SEM', 'pchembl_value_N', 
                    'pchembl_value_Median', 'pchembl_value_MAD','Quality'
                   ]

activity_reader = pd.read_csv(activity_data, 
                     sep='\t', 
                     compression='xz', 
                     chunksize = 10000, 
                     iterator=True, 
                     dtype=dtypes,
                     converters=converters,
                     usecols=activity_columns
                    )



In [ ]:
activity_columns = ['Activity_ID', 'SMILES', 'InChI_AuxInfo', 'accession', 'Protein_Type', 'AID', 'doc_id',
                    'Year', 'type_IC50', 'type_EC50', 'type_KD', 'type_Ki', 'relation', 'pchembl_value',
                    'pchembl_value_Mean', 'pchembl_value_StdDev', 'pchembl_value_SEM', 'pchembl_value_N', 
                    'pchembl_value_Median', 'pchembl_value_MAD','Quality', 'all_doc_ids', 'all_years'
                   ]

activity_reader = pd.read_csv(activity_data, 
                     sep='\t', 
                     compression='xz', 
                     chunksize = 100, 
                     iterator=True, 
                     dtype=dtypes,
                     converters=converters,
                     usecols=activity_columns
                    )

for i,df in enumerate(activity_reader):
    df.head()
    if i>0:
        break
    

In [ ]:
xlen = lambda x: (len(x))

df['pchembl_len'] = df['pchembl_value'].map(xlen)

In [ ]:
df.head()

In [ ]:
activity_type_map = {
        '1000':'IC50',
        '0100':'EC50',
        '0010':'KD',
        '0001':'Ki',
        '0000':'other',
    }
   

# df['atype'] = df.apply(lambda x: get_atype(f"{x['type_IC50']}{x['type_EC50']}{x['type_KD']}{x['type_Ki']}"), axis=1)

multis = df[df['pchembl_len']>1]

In [ ]:
multis.head()

In [ ]:
def sanitize_and_split(row, length, spl=';'):
    split = [v.rstrip() for v in str(row).split(spl)]
    if len(split)!= length:
        split = [split[0] for i in range(0,length)]
    
    split = [None if x == '' else x for x in split]
    
    return split

def get_activity_dicts(row):
    slice_list = []
    pchembl_values = [v.rstrip() for v in row.pchembl_value]

    aids = sanitize_and_split(row=row.AID,length=row.pchembl_len)        
    doc_ids = sanitize_and_split(row=row.all_doc_ids,length=row.pchembl_len)
    years = sanitize_and_split(row=row.all_years,length=row.pchembl_len)
    type_IC50s = sanitize_and_split(row=row.type_IC50,length=row.pchembl_len)         
    type_EC50s = sanitize_and_split(row=row.type_EC50,length=row.pchembl_len)
    type_KDs = sanitize_and_split(row=row.type_KD,length=row.pchembl_len)
    type_Kis = sanitize_and_split(row=row.type_Ki,length=row.pchembl_len)

    for j in range(0, row.pchembl_len):
        update_dict = {
            'pchembl_value': pchembl_values[j],
            'AID': aids[j],
            'doc_id': doc_ids[j],
            'Year': years[j],
            'type_IC50': type_IC50s[j],
            'type_EC50': type_EC50s[j],
            'type_KD': type_KDs[j],
            'type_Ki': type_Kis[j]
        }
        row_copy = copy(row._asdict())

        row_copy.update(update_dict)

        slice_list.append(row_copy)
            
    return slice_list

In [ ]:
updated_rows = []
for row in multis.itertuples():
    updated_rows.extend(get_activity_dicts(row))

In [ ]:
sval = lambda x: (x[0])

multi_expanded = pd.DataFrame(updated_rows)
singles = df[df['pchembl_len']==1]
singles['pchembl_value'] = singles['pchembl_value'].map(sval)
all_expanded = pd.concat([multi_expanded, singles])

In [ ]:
all_expanded['atype'] = all_expanded.apply(lambda x: get_atype(f"{x['type_IC50']}{x['type_EC50']}{x['type_KD']}{x['type_Ki']}"), axis=1)

In [ ]:
all_expanded

In [ ]:
l = session.query(
 Molecule.connectivity,
 Molecule.fp,
 Molecule.id,
 Molecule.inchi,
 Molecule.inchi_auxinfo,
 Molecule.inchi_key,
 Molecule.mol,
 Molecule.smiles
                 
                 ).filter_by(smiles='Nc1ncnc2[nH]c(SCCOc3ccc(Cl)cc3)nc12', inchi_auxinfo='"AuxInfo=1/1/N:12,15,11,16,8,7,20,13,10,3,2,18,5,14,1,21,19,4,17,9,6/E:(1,2)(3,4)/rA:21NCCNCSCCOCCCCClCCNCNCN/rB:s1;d2;s3;d4;s5;s6;s7;s8;s9;d10;s11;d12;s13;s13;s10d15;s5;s3s17;d18;s19;s2d20;/rC:;;;;;;;;;;;;;;;;;;;;;"').all()
[r._asdict() for r in l]

In [ ]:
def process_row(row):
    
    activity_type_map = {
        '1000':'IC50',
        '0100':'EC50',
        '0010':'KD',
        '0001':'Ki',
        '0000':'other',
    }
    
    mkey = f'{row.SMILES}_{row.InChI_Auxinfo}'
    mol_ids = mol_ids[mkey]
    qid = qualities[row.Quality]

    qc,quality = get_or_instance(session=session, model=Quality, quality=row.Quality)
    if qc: rows.append(quality)
    qid = quality.id

    tc,target_id = get_or_instance(session=session, model=Protein, target_id=row.target_id)
    if tc: rows.append(target_id)
    tid = target_id.target_id

    molecule_id = molecule.id

    slice_list = get_activity_dicts(row)

    for s in slice_list:

        a = f"{s['type_IC50']}{s['type_EC50']}{s['type_KD']}{s['type_Ki']}"
        activity_type_str = activity_type_map[a]

        activity_type = get_or_create(session=session, model=ActivityType, type=activity_type_str).id

        try:
            y = int(s['Year'])
        except:
            y = None

        if str(s['doc_id']) in ['nan', 'NaN']:
            doc_id = None
        else:
            doc_id = str(s['doc_id'])

        activity = Activity(
            papyrus_activity_id=s['Activity_ID'],
            quality=qid,
            target_id=tid,
            molecule_id = molecule_id,
            accession=s['accession'],
            protein_type=s['Protein_Type'],
            aid = s['AID'],
            doc_id = doc_id,
            year = y,
            type = activity_type, 
            relation = s['relation'],
            pchembl_value = s['pchembl_value'],
            pchembl_value_mean = s['pchembl_value_Mean'],
            pchembl_value_stdev = s['pchembl_value_StdDev'],
            pchembl_value_SEM = s['pchembl_value_SEM'],
            pchembl_value_n = s['pchembl_value_N'],
            pchembl_value_median = s['pchembl_value_Median'],
            pchembl_value_mad = s['pchembl_value_MAD'],   
        )

        rows.append(activity)
        
    return rows

In [ ]:
mol_ids

In [ ]:
count = 0
dels = []
for k in tqdm(mol_ids.keys()):
    if len(mol_ids[k])>1:
        to_del = mol_ids[k][1:]
        dels.extend(to_del)
#         for i in to_del:
#             session.query(CID).filter_by(molecule_id=i).delete()
#             session.query(Molecule).filter_by(id=i).delete()

In [ ]:
cids = []
mols = []
count = 0

def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

for group in chunker(dels, 1000):
    count +=1
    print(f'{count*1000}/{len(dels)}')
    cids.extend(session.query(CID).filter(CID.molecule_id.in_(group)).all())
    mols.extend(session.query(Molecule).filter(Molecule.id.in_(group)).all())

In [ ]:
session.commit()

In [ ]:
session.rollback()

In [ ]:
len(cids)

In [ ]:
[session.delete(cid) for cid in cids]

In [ ]:
[session.delete(m) for m in mols]

In [ ]:
session.commit()

In [ ]:
session.rollback()

In [ ]:
count = 0
for group in chunker(cids, 1000):
    count +=1
    print(f'{count*1000}/{len(cids)}')
    [session.delete(c) for c in group]
    session.commit()

In [2]:
import sys
sys.path.append('/code')

from database.models import (Protein, Organism, Classification, Molecule, Activity, ActivityType, Source, Quality, CID)

from sqlalchemy import create_engine
from sqlalchemy.orm import scoped_session
from sqlalchemy.orm import sessionmaker
import os
import glob

import pandas as pd
import numpy as np
import json

import multiprocessing
import gc
from tqdm import tqdm
from copy import copy
import numpy as np

from rdkit import Chem

def get_db_session():
    engine = create_engine(
        os.environ.get('SQLALCHEMY_URL'), convert_unicode=True,
        pool_recycle=3600, pool_size=10)
    db_session = scoped_session(sessionmaker(
        autocommit=False, autoflush=False, bind=engine))
    
    return db_session

    
mols = []
count = 0



def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))
    

# get neccessary molecule data as list of tuples
session = get_db_session()
mol_data = session.query(Molecule.id, Molecule.smiles, Molecule.inchi_auxinfo).all()
mol_ids = {}
for t in tqdm(mol_data):
    combo = f'{t[1]}_{t[2]}'
    if not combo in mol_ids.keys():
        mol_ids[combo] = [t[0]]
    else:
        mol_ids[combo].append(t[0])
        
count = 0
dels = []
for k in tqdm(mol_ids.keys()):
    if len(mol_ids[k])>1:
        to_del = mol_ids[k][1:]
        dels.extend(to_del)
        
        
for group in chunker(dels, 1000):
    count +=1
    print(f'{count*1000}/{len(dels)}')
    mols.extend(session.query(Molecule).filter(Molecule.id.in_(group)).all())



count = 0
for group in chunker(mols, 100):
    count +=1
    print(f'{count*100}/{len(mols)}')
    [session.delete(c) for c in group]
    session.commit()

/tmp/ipykernel_19519/2591433565.py:25: SADeprecationWarning: The create_engine.convert_unicode parameter and corresponding dialect-level parameters are deprecated, and will be removed in a future release.  Modern DBAPIs support Python Unicode natively and this parameter is unnecessary.
  engine = create_engine(
/tmp/ipykernel_19519/2591433565.py:45: SAWarning: relationship 'Protein.classification' will copy column protein.target_id to column ProteinClassification.protein_id, which conflicts with relationship(s): 'Classification.protein' (copies protein.target_id to ProteinClassification.protein_id), 'Protein.classifications' (copies protein.target_id to ProteinClassification.protein_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to is

1000/78698
2000/78698
3000/78698
4000/78698
5000/78698
6000/78698
7000/78698
8000/78698
9000/78698
10000/78698
11000/78698
12000/78698
13000/78698
14000/78698
15000/78698
16000/78698
17000/78698
18000/78698
19000/78698
20000/78698
21000/78698
22000/78698
23000/78698
24000/78698
25000/78698
26000/78698
27000/78698
28000/78698
29000/78698
30000/78698
31000/78698
32000/78698
33000/78698
34000/78698
35000/78698
36000/78698
37000/78698
38000/78698
39000/78698
40000/78698
41000/78698
42000/78698
43000/78698
44000/78698
45000/78698
46000/78698
47000/78698
48000/78698
49000/78698
50000/78698
51000/78698
52000/78698
53000/78698
54000/78698
55000/78698
56000/78698
57000/78698
58000/78698
59000/78698
60000/78698
61000/78698
62000/78698
63000/78698
64000/78698
65000/78698
66000/78698
67000/78698
68000/78698
69000/78698
70000/78698
71000/78698
72000/78698
73000/78698
74000/78698
75000/78698
76000/78698
77000/78698
78000/78698
79000/78698
100/78698


KeyboardInterrupt: 

In [3]:
session.rollback()

In [ ]:
count = 0
for group in chunker(mols, 100):
    count +=1
    print(f'{count*100}/{len(mols)}')
    [session.delete(c) for c in group]
    session.commit()

100/78698
200/78698
300/78698
400/78698
500/78698
600/78698
700/78698
800/78698
900/78698
1000/78698
1100/78698
1200/78698
1300/78698
1400/78698
1500/78698
1600/78698
1700/78698
1800/78698
1900/78698
2000/78698
2100/78698
2200/78698
2300/78698
2400/78698
2500/78698
2600/78698
2700/78698
2800/78698
2900/78698
3000/78698
3100/78698
3200/78698
3300/78698
3400/78698
3500/78698
3600/78698
3700/78698
3800/78698
3900/78698
4000/78698
4100/78698
4200/78698
4300/78698
4400/78698
4500/78698
4600/78698
